In [ ]:
import os
import csv
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import re
import logging

# Define base URL and main folder for saving data
BASE_URL = "https://lawphil.net/judjuris/judjuris.html"
MAIN_FOLDER = "Data/Jurisprudence"

# Define delay and maximum retries
REQUEST_DELAY = 0  # seconds between requests
TIMEOUT = 10  # seconds before timing out the request

# Configure logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

# Function to create folders if they do not exist
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Function to sanitize filenames and folder names
def sanitize_filename(name, max_length=100):
    # Replace invalid characters and truncate the name if it's too long
    sanitized_name = re.sub(r'[\\/*?:"<>|]', "_", name)
    return sanitized_name[:max_length]  # Ensure the filename doesn't exceed max_length


# Function to check if a folder already contains data
def is_data_collected(year_folder, month_folder):
    # Check if the folder contains any CSV files (indicating data has been collected)
    if not os.path.exists(month_folder):
        create_folder(month_folder)
    return any(file.endswith(".csv") for file in os.listdir(month_folder))

# Function to scrape data from each case page
def scrape_case_data(case_url, year, month, case_number):
    retries = 3
    while retries > 0:
        try:
            response = requests.get(case_url, timeout=TIMEOUT)
            response.raise_for_status()
            case_soup = BeautifulSoup(response.content, 'html.parser')
            blockquote = case_soup.find("blockquote")

            if blockquote:
                paragraphs = blockquote.find_all("p")
                if len(paragraphs) >= 3:
                    # Extract Case Number (remove square brackets), Title, and full data
                    case_number_text = re.sub(r'\[|\]', '', paragraphs[2].get_text(strip=True))
                    title_text = paragraphs[3].get_text(strip=True)
                    case_data = blockquote.get_text(separator="\n").strip()
                    return case_number_text, title_text, case_data
            return None, None, None
        except requests.RequestException as e:
            retries -= 1
            logger.warning(f"Error fetching case data from {case_url}: {e}. Retrying {retries} more times...")
            time.sleep(REQUEST_DELAY * (2 ** retries))  # Exponential backoff
            
    logger.error(f"Failed to fetch case data after retries: {case_url}")
    return None, None, None

# Function to get all links from a page and retry if request fails
def get_links(url, retry=3):
    while retry > 0:
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            return soup
        except requests.RequestException as e:
            logger.warning(f"Error fetching {url}: {e}. Retrying {retry - 1} more times...")
            retry -= 1
            time.sleep(2)  # wait before retry
    logger.error(f"Failed to fetch {url} after retries.")
    return None

# Main function to scrape years, months, and cases
def scrape_jurisprudence():
    # Fetch the base index page
    soup = get_links(BASE_URL)
    if not soup:
        logger.error("Failed to fetch the base page.")
        return
    
    # Create the main directory for storing data
    create_folder(MAIN_FOLDER)
    
    # Find all year links and sort them chronologically
    # Find all year links and filter them to start from 1997
    year_links = sorted([link for link in soup.select("a.off_n1") if int(link.get_text()) >= 1997],key=lambda link: int(link.get_text()))

    
    for year_link in year_links:
        year_text = year_link.get_text()
        year_url = urljoin(BASE_URL, year_link['href'])
        year_folder = os.path.join(MAIN_FOLDER, sanitize_filename(year_text))
        create_folder(year_folder)
        
        logger.info(f"Starting processing for year: {year_text}")
        
        # Fetch the year page
        year_soup = get_links(year_url)
        if not year_soup:
            continue
        
        # Find all month links and sort them from January to December
        month_links = sorted(year_soup.select("a.off"), key=lambda link: link.get_text())
        
        for month_link in month_links:
            month_text = month_link.get_text()
            month_folder = os.path.join(year_folder, sanitize_filename(month_text))
            
            # Check if the data for the year and month has already been collected
            if is_data_collected(year_folder, month_folder):
                logger.info(f"Data for {month_text} {year_text} already collected. Skipping.")
                continue

            logger.info(f"Starting processing for month: {month_text} of year {year_text}")
            
            # Fetch the month page
            month_url = urljoin(year_url, month_link['href'])
            month_soup = get_links(month_url)
            if not month_soup:
                continue
            
            # Find all case links on the month page
            case_links = month_soup.find_all("a")
            
            for case_link in case_links:
                # Skip links that don't have an 'href' attribute or are empty
                if 'href' not in case_link.attrs or not case_link['href']:
                    logger.warning(f"Skipping invalid link in month {month_text}, year {year_text}: {case_link}")
                    continue

                case_text = case_link.get_text()
                case_url = urljoin(month_url, case_link['href'])
                
                # Sanitize the case number text to create a valid file name
                case_file_name = sanitize_filename(case_text) + ".csv"
                csv_file_path = os.path.join(month_folder, case_file_name)
                
                logger.info(f"Processing case number: {case_text} in month {month_text}, year {year_text}")
                
                # Scrape case data
                case_number, title, case_data = scrape_case_data(case_url, year_text, month_text, case_text)
                
                if case_data:
                    # Save case data to a CSV file
                    with open(csv_file_path, mode="w", newline='', encoding="utf-8", errors="replace") as csv_file:
                        writer = csv.writer(csv_file)
                        writer.writerow(["Year", "Month", "Case Number", "Title", "Data"])
                        writer.writerow([year_text, month_text, case_number, title, case_data])
                    
                    logger.info(f"Saved case data for {case_text} in month {month_text}, year {year_text}")
                else:
                    logger.warning(f"Skipping case number: {case_text} as no data was found")

                # Respectful delay to avoid hitting the server too hard
                time.sleep(1)

    logger.info("Completed scraping process.")

# Run the scraping process
scrape_jurisprudence()


[2024-11-15 21:23:00,647] INFO: Starting processing for year: 1997
[2024-11-15 21:23:00,915] INFO: Starting processing for month: April of year 1997
[2024-11-15 21:23:01,280] WARNING: Skipping invalid link in month April, year 1997: <a name="top"></a>
[2024-11-15 21:23:01,280] INFO: Processing case number: G.R. No. 110286 in month April, year 1997
[2024-11-15 21:23:01,602] INFO: Saved case data for G.R. No. 110286 in month April, year 1997
[2024-11-15 21:23:02,607] WARNING: Skipping invalid link in month April, year 1997: <a class="vs">vs.</a>
[2024-11-15 21:23:02,608] INFO: Processing case number: G.R. No. 116732 in month April, year 1997
[2024-11-15 21:23:02,984] INFO: Saved case data for G.R. No. 116732 in month April, year 1997
[2024-11-15 21:23:03,987] WARNING: Skipping invalid link in month April, year 1997: <a class="vs">vs.</a>
[2024-11-15 21:23:03,987] INFO: Processing case number: A.M. No. MTJ-97-1114 in month April, year 1997
[2024-11-15 21:23:04,281] INFO: Saved case data f